Competition rules from fivethirtyeight:

https://fivethirtyeight.com/features/can-you-rule-riddler-nation/

In a distant, war-torn land, there are 10 castles. There are two warlords: you and your archenemy. Each castle has its own strategic value for a would-be conqueror. Specifically, the castles are worth 1, 2, 3, …, 9, and 10 victory points. You and your enemy each have 100 soldiers to distribute, any way you like, to fight at any of the 10 castles. Whoever sends more soldiers to a given castle conquers that castle and wins its victory points. If you each send the same number of troops, you split the points. You don’t know what distribution of forces your enemy has chosen until the battles begin. Whoever wins the most points wins the war.

Submit a plan distributing your 100 soldiers among the 10 castles. Once I receive all your battle plans, I’ll adjudicate all the possible one-on-one matchups. Whoever wins the most wars wins the battle royale and is crowned king or queen of Riddler Nation!

In [ ]:
from functools import reduce
from random import random, randint, sample, shuffle
from operator import add
import numpy as np
import math

In [ ]:
N_castles = 10
N_soldiers = 100
population_count = 100

In [ ]:
class rule_riddler_nation:
    def __init__(self, N_castles, N_soldiers, population_count):
        self.N_castles = N_castles
        self.N_soldiers = N_soldiers
        self.population_count = population_count

    def individual(self, version="regular"):
        """Create individual random arrangement of soldiers per castle."""
        
        if version == "regular" and self.N_soldiers % self.N_castles == 0:
            values = [self.N_soldiers/self.N_castles for _ in range(self.N_castles)]
            values = self.mutate_individual(values, times=10)
        else:
            values = []
            aux_sum = self.N_soldiers
            n = self.N_castles
            while n > 1:
                value = randint(0, aux_sum)
                values.append(value)
                aux_sum -= value
                n -= 1
            values.append(aux_sum)
            shuffle(values)
            
        return values
    
    def mutate_individual(self, individual, times=5):
        """Move soldiers around from a given arrangement."""
        individual = individual[:]
        for _ in range(times): 
            index_1, index_2 = sample(range(self.N_castles), 2)
            how_much = sample(range(int(individual[index_1]+1)), 1)[0]
            individual[index_1] -= how_much
            individual[index_2] += how_much
        return individual
    
    def population(self):
        """Create population of individual arrangements."""
        return [ self.individual() for x in range(self.population_count)]
    
    def match(self, individual_1, individual_2):
        """Or battle. Check which arrangement scores higher."""
        score_1 = 0
        score_2 = 0
        for i, (i1, i2) in enumerate(zip(individual_1, individual_2)):
            if i1 > i2:
                score_1 += i
            elif i2 > i1:
                score_2 += i
            else:
                score_1 += i*0.5
                score_2 += i*0.5
        return (score_1 >= score_2, score_2 >= score_1)

    def score_population(self, population):
        """Run matches for all pairs in population."""
        scores = dict((i,0) for i in range(len(population)))
        for index_p1, p1 in enumerate(population):
            for index_p2, p2 in enumerate(population[1:]):
                score1, score2 = self.match(p1, p2)
                scores[index_p1] += score1
                scores[index_p2] += score2
        return [(score, population[i]) for i, score in scores.items()]
    
    def score_individual_with_population(self, individual, population):
        """Cross validate individual with given population."""
        score = 0
        for index_p, p in enumerate(population):
            score_i, _ = self.match(individual, p)
            score += score_i
        return score
    
    def evolve(self, pop, retain=0.25, random=0.75):
        """Evolve population choosing best performers and add random individuals."""
        grades = self.score_population(pop)
        grades.sort(key=lambda x: -x[0])
        best_performers = [elem[1] for elem in grades[:math.floor(retain*len(pop))]]
        print('best_performer', best_performers[0])
        new_random = [self.individual() for p in range(math.floor(random*len(pop)))]    
        return best_performers + new_random

In [ ]:
rrn = rule_riddler_nation(N_castles, N_soldiers, population_count)
pop = rrn.population()
for _ in range(100):
    pop = rrn.evolve(pop)